In [1]:
import pickle5 as pickle
import random
import numpy as np

with open('../tokens', 'rb') as pickle_file:
    tokenized_sent = pickle.load(pickle_file)

random.shuffle(tokenized_sent)

def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

test_tokens = tokenized_sent[:10000]
train_tokens = [' '.join(token[2]) for token in tokenized_sent[10000:]]

In [2]:
!pip3 install --upgrade tensorflow-gpu
# Install TF-Hub.
!pip3 install tensorflow-hub

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

2022-04-18 18:12:55.986869: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-18 18:12:55.986888: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model = hub.load(module_url)
print ("module %s loaded" % module_url)

2022-04-18 18:13:01.593511: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-18 18:13:01.593529: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-18 18:13:01.593666: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:163] no NVIDIA GPU device is present: /dev/nvidia0 does not exist
2022-04-18 18:13:01.594086: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [5]:
sentence_embeddings = model(train_tokens)

2022-04-18 18:19:41.574833: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 19993262080 exceeds 10% of free system memory.


In [6]:
query = ' '.join(test_tokens[0][2])
query_vec = model([query])[0]

In [ ]:
for sent in train_tokens[:10]:
  sim = cosine(query_vec, model([sent])[0])
  print("Sentence = ", sent, "; similarity = ", sim)

In [7]:
import numpy as np
import pandas as pd
results = []

sentences = [' '.join(token[2]) for token in tokenized_sent]
themes = [token[1] for token in tokenized_sent]

In [8]:
tokens_embedding = model(sentences) 

2022-04-18 18:20:23.549532: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 24142300160 exceeds 10% of free system memory.


In [9]:
end_tokens = False

gap = 5000
offset = 0
limit = gap
length = len(tokens_embedding)


first_loop = True

In [10]:
while not end_tokens:

    if limit > length:
        limit = length
        end_tokens = True
    
    sub_tokens_embedding = tokens_embedding[offset:limit]
    results = []
    for index, token in enumerate(sub_tokens_embedding):
        row = np.append(token, themes[index + offset])
        results.append(row)

    df = pd.DataFrame.from_dict(results)

    offset += gap
    limit += gap
    
    df.to_csv('/app/Embedding/dataframes/universal_sentence_python.csv', index=False, encoding='utf-8', mode='a', header=False)